In [ ]:
import csv
import urllib.request
import json
import numpy as np
import matplotlib.pyplot as plt
from urllib.parse import quote
import os
import re

In [ ]:
# Moving the characters from our .csv file into a list of tuples
characters = []

with open("HP_characters.csv", "r", encoding="utf8") as sent_file:
    csv = csv.reader(sent_file, delimiter=",")
    for row in csv:
        name = row[0].replace(' ', '_') 
        # Parentage, House, and Occupation will be our attributes in the nodes
        parentage = row[1]
        house = row[2]
        occupation = row[3]
        characters.append((name, parentage, house, occupation))

In [ ]:
### Do not need to run this, might just take it out or place it somewhere else????
lengths = []

#https://www.reddit.com/r/learnpython/comments/muwu7v/scraping_fandomwiki_pages/
baseurl = "https://harrypotter.fandom.com/api.php?"
action = "action=query"

for character in characters:
    #if (character[0] == 'Eldred_Worple'):
    # Set up the query for the character
    title = "titles=" + character[0]
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json"
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)

    # Since we have checked the articles, we know that urlopen will succeed
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    lengths.append((character, len(wikitext)))

    try:
        jsonobj = json.loads(wikitext)
    except ValueError: 
        print(f'Decoding JSON has failed for {character}, moving on...')
    # Get the number for the article
    num = list(jsonobj['query']['pages'].keys())[0]
    # Get the wikitext
    wikitext = jsonobj['query']['pages'][num]['revisions'][0]['*']
    f_name = character[0] + '.txt'

    # Remove <ref> tags including content.
    clean_text = re.sub(r'<ref.*?</ref>', "", wikitext)
    clean_text = re.sub(r'<ref.*?/>', "", clean_text)

    # Write to a file with that name
    with open("characters/" + f_name, 'w') as f:
        f.write(clean_text)

In [ ]:
def find_aliases(text, alias_dict, name):
    #print("Name: " + name)
    #Find all aliases. Will be between |alias and |title, can be on several lines
    aliases = re.findall(r'\|alias = (.(.|\s)*?)\|', text)
    
    if len(aliases)==0:
        return
    else:
        #Take the first match in the first group and split by new line so each alias becomes an element
        all_aliases = aliases[0][0].split("\n")
        #print("length of aliases: " + str(len(all_aliases)))

        #Find the aliases to keep
        for alias in all_aliases:
            #Don't keep the ones used for a disguise
            if "disguise" in alias:
                continue
            #Mudblood not an alias
            if "Mudblood" in alias:
                continue
            #Don't keep the ones used to tell others a wrong name 
            if "the name he told" in alias:
                continue
            if "the name she told" in alias:
                continue
            #If alias is empty string then don't keep
            if alias == '':
                continue
            #Cleaning of the aliases
            processed_alias = alias.split(" (by")[0]
            if '{{' in processed_alias:
                processed_alias = alias.split("{{")[0]
            processed_alias = processed_alias.replace('*', '')
            processed_alias = processed_alias.replace('[[', '').replace(']]', '').replace('"', '')
            if processed_alias in alias_dict:
                print("Processed alias: " + processed_alias + ", belonging to: " + name)
            else:
                alias_dict[processed_alias.lower()] = name   

In [ ]:
alias_dictionary = {}
not_added = []
for file_ in os.listdir("./characters"):
    #print(file_)
    name = file_[:-4]
    #print(name)
    with open("./characters/"+file_, 'r') as f:
        text = f.read()
    first_name = name.split("_")[0].lower()
    #print("First name: " + first_name)
    full_name = name.replace('_', ' ').lower()
    if first_name in alias_dictionary:
        not_added.append((first_name, name))
    else:
        alias_dictionary[first_name] = name
    alias_dictionary[full_name] = name
    find_aliases(text, alias_dictionary, name)
    


In [ ]:
print(not_added)

In [ ]:
for item in not_added:
    print("Dict has: " + alias_dictionary[item[0]] + " instead of: " + item[1])
    

In [ ]:
alias_dictionary['james'] = 'James_Potter_I'
alias_dictionary['frank'] = 'Frank_Longbottom'
alias_dictionary['marcus'] = 'Marcus_Flint'
alias_dictionary['hermione'] = 'Hermione_Granger'
alias_dictionary['mr. dursley'] = 'Vernon_Dursley'
alias_dictionary['mrs. dursley'] = 'Petunia_Dursley'
alias_dictionary['mcgonagall'] = 'Minerva_McGonagall'
alias_dictionary['dumbledore'] = 'Albus_Dumbledore'

In [ ]:
print(alias_dictionary['ron'])
print(alias_dictionary['harry potter'])
print(alias_dictionary['frank'])
print(alias_dictionary['the boy who lived'])
print(alias_dictionary[' ab '])
p

In [ ]:
if 'mr' in alias_dictionary:
    del alias_dictionary['mr']
if 'mrs' in alias_dictionary:
    del alias_dictionary['mrs']
if 'fat' in alias_dictionary:
    del alias_dictionary['fat']

for alias, name in alias_dictionary.items():
    if (len(alias)<4):
        #print(alias)
        alias_dictionary[' '+alias+' '] = alias_dictionary.pop(alias)

In [ ]:
#print(sorted(alias_dictionary.items()))
print(alias_dictionary[' al '])

### Notes to aliases/ first names that collapses:
**Following characters only appears in the last chapter of B7:**
- James Potter II 
- Edward Lupin
- Rose Granger-Weasley
- Lily L. Potter

- Frank Bryce only appears in first chapter of B4

For the above characters we will change dictionary when looking in the corresponding chapters

**Following characters has first names that are the same and appears in different books:**
- Marcus Flint (appears in B1, B2, B3) and Marcus Belby (appears in B6), equally important
- Graham Montague (in B3, B5) more important than Graham Pritchard (in B4)
- Frank Longbottom (in B5) more important than Frank Bryce (in B4)
- Avery II (in B4, B5) more important than Avery I (in B6)

**Following characters has first names that are the same and appears in the same books:**
- Graham Montague (in B3, B5) more important than Graham Montague's father (in B5) and mother (in B5)
- Hermione Granger (in all books) more important than Hermione Granger's Father(in B2, B5) and Hermione Granger's Mother (in B2, B5)
- Dennis Creevey (in B4-B6) more important than Dennis (in B5)
- Ernest Macmillan (in B2, B4-B7) more important than Ernest Prang (in B3, B6)
- Fat Friar (in B1, B2, B5) and Fat Lady (in all books) equally important
- Mary Cattermole (in B7) equally important as Mary Macdonald (in B7)
- Evans sister's father (in B7) equally important as Evans sister's mother (in B7)
- Ludovic Bagman (in B4) more important than Ludovic Bagman's Goblin creditors (in B4)

For those not in the same books we will make dictionary to fit with book. 
For those in the same books we will have to find another solution and prioritize the most important characters. 

Mr. and Mrs. Dursley appears often like this. Needs to be replaced with Vernon_Dursley and Petunia_Dursley.
The Potters should be replaced with James_Potter_I and Lily_J._Potter



In [ ]:
# Replace names in chapters so they are called our character names and not aliases

def replace_aliases(text, dst_file):
    clean_text = text
    clean_text = clean_text.replace('\t', ' ').replace('\n', ' ').replace('  ', ' ')
    clean_text = clean_text.lower()
    clean_text = clean_text.replace('mr. and mrs. dursley', 'Vernon_Dursley and Petunia_Dursley')
    clean_text = clean_text.replace('the potters', 'James_Potter_I and Lily_J._Potter')

    for alias, name in alias_dictionary.items():
        clean_text = clean_text.replace(alias, name)

     # Write to a file with that name
    with open(dst_file, 'w') as f:
            f.write(clean_text)

In [ ]:
# Loop over chapters, will do it book by book as we have some places where we need to change dictionary

#Book 1
if 'rose' in alias_dictionary:
    del alias_dictionary['rose']
for chap in os.listdir("./B1"):
    text = ''
    with open('./B1/'+chap, 'r') as f:
            text = f.read()
            f.close()
    filename = chap.split('_')[1].lower()
    dst_filename = './B1/replaced_' + filename
    replace_aliases(text, dst_filename)

In [ ]:
# Only appearing in book 5
alias_dictionary['rose'] = "Rose_Zeller"